# 필요한 라이브러리 import

In [183]:
import os
import json
import numpy as np
import pandas as pd
from gensim import corpora
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as spr
import configparser
import pymysql
from itertools import chain
from collections import defaultdict,Counter
import hnswlib
import warnings
import tqdm
import pickle
import matplotlib.pyplot as plt
from implicit.als import AlternatingLeastSquares
from sklearn.decomposition import NMF
from itertools import repeat
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [107]:
with open('./data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
song_df = pd.DataFrame.from_dict(song_dict)

In [108]:
with open('./data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)

In [3]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)

In [96]:
train_df.head()

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,절삭태그,절삭태그아이디,태그수
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[락],[25304],1
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000,"[추억, 회상]","[17676, 152]",2
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000,"[까페, 잔잔한]","[21656, 26271]",2
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 겨울노래, 크리스마스, 겨울왕국]","[19772, 22488, 10907, 19970, 8380, 21963, 170,...",8
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000,[댄스],[13941],1


# 태그-id간 동기화를 위한 데이터베이스 연결

In [4]:
# config parser 객체 생성
config = configparser.ConfigParser()

# parser객체로 config.ini 파일 읽기
config.read('config.ini')

# 변수저장

user = config['DEFAULT']['ADMIN_USER_NAME']
passwd = config['DEFAULT']['ADMIN_PASSWORD']
host = config['DEFAULT']['RDS_ENDPOINT']
port = config['DEFAULT']['PORT']
database = config['DEFAULT']['DEFAULT_DATABASE']

In [5]:
melon_recom_db = pymysql.connect(
    host = host,
    user = user,
    password = passwd,
    database = database,
    port = int(port),
    cursorclass = pymysql.cursors.DictCursor
)

In [6]:
melon_cursor = melon_recom_db.cursor()
melon_cursor.execute('select * from tag')
tag_db_list=melon_cursor.fetchall()

# tag 딕셔너리 만들기 

In [7]:
tag_to_id = {}
id_to_tag = {}

for tag_db in tag_db_list:
    tag_to_id[tag_db['tag']] = tag_db['tag_id']
    
for tag_db in tag_db_list:
    id_to_tag[tag_db['tag_id']] = tag_db['tag']

# tag 전처리 빈도수로 컷

In [8]:
# train dataframe tag 컬럼의 모든 tag들 (중복포함)
tags_all = train_df['tags'].tolist()

# 태그의 빈도수를 가진 dict, Counter 써도 됨
tags_frequency = defaultdict(int)

# 특정 tag가 나올 때마다 1더하기
for tags in tags_all:
    for tag in tags:
        tags_frequency[tag] += 1

In [9]:
tag_freq = pd.DataFrame().from_dict(tags_frequency,orient="index")

In [10]:
tag_freq.columns=['빈도']

In [11]:
tag_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
빈도,29160.0,16.335082,247.011075,1.0,1.0,1.0,3.0,16465.0


In [12]:
def filter_func(x):
    temp = []
    for tag in x:
        if tags_frequency[tag] > 20:
            temp.append(tag)
        else:
            pass
    return temp
            

train_df['tag_cut'] = train_df['tags'].map(filter_func)

In [13]:
train_df['new_tags_cut_id'] = train_df['tag_cut'].map(lambda x : [tag_to_id[v] for v in x])
train_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tag_cut,new_tags_cut_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[락],[25304]


## 컬럼명 한글화

In [14]:
train_df.columns=['태그','플리아이디','플리제목','노래들','좋아요수','변경일자','절삭태그','절삭태그아이디']

In [16]:
train_df['태그수'] = train_df['절삭태그'].map(len)

In [17]:
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,절삭태그,절삭태그아이디,태그수
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[락],[25304],1


# Train Test Split

In [242]:
# 태그가 10개인거 부터 train에 쓸거임
origin_tags = train_df[train_df['태그수']>9]['절삭태그'].tolist()

train_tags = []
test_tags = []

# 태그는 많아봐야 3개 사분위수 75% -> 3개
for tags in origin_tags:

    train_tags.append(tags[:2])
    test_tags.append(tags[2:])

In [243]:
cal_tag_hit_df = pd.DataFrame(columns=['태그원본','예측용태그','검증용태그','als_예측결과','svd_예측결과','als_히트','svd_히트'])
cal_tag_hit_df['태그원본'] = origin_tags

In [244]:
cal_tag_hit_df['예측용태그'] = train_tags
cal_tag_hit_df['검증용태그'] = test_tags
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,als_예측결과,svd_예측결과,als_히트,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브]","[Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌드, 일렉]",NaN,NaN,NaN,NaN
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카]","[매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일렉, 신나는]",NaN,NaN,NaN,NaN
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[잔잔한, 감성]","[새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]",NaN,NaN,NaN,NaN
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[휴식, 새벽]","[펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]",NaN,NaN,NaN,NaN
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[잠, 매장]","[매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]",NaN,NaN,NaN,NaN


# 희소행렬생성 태그,음악 행렬

In [105]:
song_df.colums=['세부장르','발매일','앨범명','앨범ID','가수ID','노래명','대장르','가수명','노래ID']

In [106]:
song_df.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [111]:
genre_dict['GN9000'] = '기타장르'

genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v

genre_detail_dict = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        genre_detail_dict[k] = new_value

In [112]:
genre_detail_dict['GN0901']

'POP_세부장르전체'

In [132]:
genre_detail_id_to_code = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_id_to_code[i] = v

In [133]:
genre_detail_code_to_id = {}

for i,v in enumerate(list(genre_detail_dict.keys())):
    genre_detail_code_to_id[v] = i

In [134]:
song_genre_detail_dict = defaultdict(list)

for codes,id in zip(song_df['song_gn_dtl_gnr_basket'].tolist(),song_df['id'].tolist()):
    for code in codes:
        song_genre_detail_dict[id].append(genre_detail_code_to_id[code])

[113, 118]

In [156]:
def fetcher(x):
    temp = []
    for song in x:
        genre_ids = song_genre_detail_dict[song]
        for id in genre_ids:
            temp.append(id)
        
    return temp

train_df['세부장르'] = train_df['노래들'].map(fetcher)
train_df.head(1)

,태그,플리아이디,플리제목,노래들,좋아요수,변경일자,절삭태그,절삭태그아이디,태그수,세부장르
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,[락],[25304],1,"[97, 96, 48, 49, 56, 67, 60, 56, 68, 48, 49, 5..."


In [217]:
genre_id_merge = defaultdict(list)

for ids,genres in zip(train_df['절삭태그아이디'].tolist(),train_df['세부장르'].tolist()):
    for id in ids:
        genre_id_merge[id].extend(genres)

In [218]:
for k,v in genre_id_merge.items():
    genre_id_merge[k] = dict(Counter(v))

In [219]:
for k,value_dict in genre_id_merge.items():
    sum_val = sum(value_dict.values())
    for key,value in value_dict.items():
        value_dict[key] = np.round(((value/sum_val)*100),3)

In [223]:
genre_id_merge[25304]

{97: 0.866,
 96: 1.624,
 48: 5.193,
 49: 2.605,
 56: 14.977,
 67: 1.959,
 60: 0.674,
 68: 5.89,
 58: 1.735,
 91: 0.257,
 88: 0.848,
 89: 0.592,
 51: 0.033,
 156: 0.787,
 148: 0.064,
 145: 0.817,
 71: 0.322,
 70: 0.765,
 27: 7.689,
 31: 3.454,
 0: 2.962,
 32: 3.268,
 9: 0.288,
 5: 0.686,
 33: 0.062,
 4: 1.088,
 202: 0.458,
 201: 0.759,
 206: 0.316,
 203: 0.295,
 207: 0.473,
 209: 0.368,
 208: 0.08,
 103: 1.392,
 106: 0.428,
 7: 0.079,
 8: 0.188,
 18: 5.318,
 20: 3.974,
 26: 3.292,
 61: 1.539,
 73: 0.249,
 72: 0.118,
 214: 0.108,
 105: 0.53,
 94: 0.138,
 79: 0.423,
 62: 1.407,
 146: 0.153,
 128: 0.738,
 132: 0.02,
 147: 0.515,
 133: 0.115,
 25: 1.863,
 66: 0.447,
 57: 0.434,
 69: 0.323,
 63: 0.243,
 90: 0.138,
 100: 0.134,
 15: 0.622,
 17: 0.398,
 16: 0.139,
 47: 0.692,
 19: 0.779,
 43: 1.221,
 204: 0.045,
 10: 0.689,
 55: 0.109,
 3: 0.933,
 129: 0.268,
 80: 0.14,
 205: 0.111,
 12: 0.438,
 11: 0.08,
 93: 0.032,
 2: 0.229,
 30: 0.564,
 24: 0.132,
 104: 0.159,
 111: 0.069,
 108: 0.094,
 15

In [224]:
row = []
col = []
dat = []

for k,v in genre_id_merge.items():
    for vk,vv in v.items():
        row.append(k)
        col.append(vk)
        dat.append(vv)

In [225]:
len(genre_id_merge)

1532

In [227]:
len(set(list(chain.from_iterable(train_df['절삭태그'].tolist()))))

1532

In [228]:
A = spr.csr_matrix((dat, (row, col)), shape=(29160, 224))

In [264]:
spr.save_npz('./data/tag_genred_csr.npz',A)

In [229]:
A = A.astype(float)

# Vectorization with als

In [52]:
als = AlternatingLeastSquares(factors=100,num_threads=6,iterations=50)

In [53]:
als.fit(A*40)

  0%|          | 0/50 [00:00<?, ?it/s]

In [54]:
with open('./tag_als_model.pickle','wb') as f:
    pickle.dump(als, f)

In [55]:
with open('./tag_als_model.pickle','rb') as f:
    als_model = pickle.load(f)

In [56]:
# 락과 관련된 태그 10개

tag_vectors = als_model.item_factors

In [57]:
data_len,dim = tag_vectors.shape

In [58]:
als_p = hnswlib.Index(space='cosine', dim=dim)  

In [59]:
als_p.init_index(max_elements=data_len, ef_construction=100, M=100)

In [60]:
als_p.add_items(tag_vectors,np.arange(data_len))

In [61]:
labels, distances = als_p.knn_query(tag_vectors[3113], k = 11)
print(labels, distances)

[[28611 27884 27634 28010 29043 28065 28004 27909 28151 28850 28143]] [[0.3833024  0.4172771  0.49452704 0.5098832  0.531496   0.5356837
  0.54083335 0.5550363  0.56858957 0.57836306 0.58445144]]


In [62]:
[id_to_tag[x] for x in labels[0][1:]]

['좋아',
 '위너',
 '임영웅',
 '몽환적인노래',
 '아이돌탐구생활',
 '방탄소년단명곡',
 'TVN',
 '고요한',
 '감미로움',
 '내적댄스유발']

# Vectorization with SVD

## SVD 만들기

In [230]:
u,s,vt = spr.linalg.svds(A,k=100)

In [231]:
svd_vectors = u

In [232]:
svd_vectors.shape

(29160, 100)

In [233]:
data_len,dim = svd_vectors.shape

In [234]:
svd_p = hnswlib.Index(space='cosine', dim=dim)  

In [235]:
svd_p.init_index(max_elements=data_len, ef_construction=100, M=100)

In [236]:
svd_p.add_items(svd_vectors,np.arange(data_len))

In [252]:
#pickle.dump(p,'./data/svd_knn_model.pickle')
with open('./data/svd_knn_model.pickle', 'wb') as f:
    pickle.dump(svd_p,f)

In [253]:
with open('./data/svd_tag_vectors.pickle', 'wb') as f:
    pickle.dump(svd_vectors,f)

In [73]:
with open('./data/svd_knn_model.pickle', 'rb') as f:
    svd_p = pickle.load(f)

with open('./data/svd_knn_model.pickle', 'rb') as f:
    p = pickle.load(f)

In [147]:
tag_to_id['백예린'] , tag_to_id['아이유']

(3113, 10095)

In [239]:
labels, distances = svd_p.knn_query(svd_vectors[3113], k = 11)
print(labels, distances)

[[ 3113  3308  6735  1611 26010 10767  1715 25523 26257 17420 12817]] [[-1.1920929e-07  3.2853001e-01  3.3607537e-01  3.9810312e-01
   4.0170991e-01  4.1195774e-01  4.2504132e-01  4.2938858e-01
   4.5882237e-01  4.8292696e-01  4.8351216e-01]]


In [240]:
#id_to_tag[24585]

[id_to_tag[x] for x in labels[0][1:]]

['국내RnB', 'RnB소울', '가을노래', '겨울밤', '국내알앤비', 'R', '그냥', '폴킴', '겨울감성', '꿀성대']

In [241]:
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,als_예측결과,svd_예측결과,als_히트,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브]","[Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌드, 일렉]","[임영웅, 에너지, 질주, 트랩, 아이돌그룹, 노동요, 아이돌탐구생활, 자연, 아이...","[헝가리, 명상_음악, 저기압, 블루지한, 세엑스왜안와, 힙합라운지, 싱스트리트_존...",0,0
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카]","[매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일렉, 신나는]","[방탄소년단명곡, 아이돌탐구생활, 매장음악, 에너지, 장르, 힙스터, 임영웅, 위너...","[여신, 치카치카, 장기휴식, Latin, blueming, 세탁, 찬송, 박효신추...",1,0
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[잔잔한, 감성]","[새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[임영웅, 좋아, 감미로움, 과거, 아이돌탐구생활, 고요한, 아이돌그룹, 괜찮아, ...","[쟁글팝, 멜론티켓, 타이가, 옛날팝송, 내려놓기, 2019년기대주, 아레나록, 매...",0,0
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[휴식, 새벽]","[펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[좋아, 감미로움, 아이돌탐구생활, 아이돌그룹, 괜찮아, 마무리, 가사로듣는, 장르...","[결정의순간, 맛있는노래, 치카치카, 만화카페, 코다라인내한, 선데이, 찬송, 텔레...",0,0
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[잠, 매장]","[매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[고요한, 졸릴때, 세련된, 매장음악, 마무리, 아이돌그룹, 좋아, 질주, 밤에듣기...","[seapunk, 심야드라이브, 모즈다이브, 기상후, 조장혁, 여자가수만, 윤석철트...",1,0


In [259]:
def tag_svd(x):
    target_ids = [tag_to_id[t] for t in x]
    
    vectors = np.zeros((100,1))
    
    for id in target_ids:
        vectors = vectors+svd_vectors[id]
    
    labels, distances = svd_p.knn_query(vectors/len(target_ids), k = 12)
    
    ids = [l for l in labels[0][2:]]
    
    return [id_to_tag[tag] for tag in ids]

In [260]:
cal_tag_hit_df['svd_예측결과'] = cal_tag_hit_df['예측용태그'].map(tag_svd)
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,als_예측결과,svd_예측결과,als_히트,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브]","[Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌드, 일렉]",NaN,"[운동할때, 드라이브, 여름, 신나는음악, 신나는, 흥, 헬스, 신나는노래, 다이어...",NaN,0
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카]","[매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일렉, 신나는]",NaN,"[일렉트로닉팝, ElectronicPop, electronica, tropical,...",NaN,0
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[잔잔한, 감성]","[새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]",NaN,"[쓸쓸, 밤, 새벽감성, 새벽, 우울, 카페, 가을, 휴식, 혼자, 따뜻한]",NaN,1
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[휴식, 새벽]","[펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]",NaN,"[휴식, 잔잔한, 힐링, 가을, 비오는날, 산책, 사랑, 까페, 잠안올때, 설렘]",NaN,0
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[잠, 매장]","[매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]",NaN,"[카페, 일상, 날씨, 잠안올때, 휴식, 생각, 잔잔한, 봄날, 편안한, 주말]",NaN,1


In [261]:
val_tag = cal_tag_hit_df['검증용태그'].tolist()
svd_tag = cal_tag_hit_df['svd_예측결과'].tolist()

In [262]:
svd_hit = []

for val,svd in zip(val_tag,svd_tag):
    if len(val) == len(set(val)-set(svd)):
        svd_hit.append(0)
    else:
        svd_hit.append(1)
             
svd_hit[:5]

[0, 0, 1, 0, 1]

In [249]:
sum(svd_hit)

cal_tag_hit_df['svd_히트'] = svd_hit

In [250]:
cal_tag_hit_df.head()

,태그원본,예측용태그,검증용태그,als_예측결과,svd_예측결과,als_히트,svd_히트
0,"[운동, 드라이브, Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌...","[운동, 드라이브]","[Pop, 트로피컬하우스, 힐링, 기분전환, 2017, 팝, 트렌드, 일렉]",NaN,"[스트레스, 운동할때, 드라이브, 여름, 신나는음악, 신나는, 흥, 헬스, 신나는노...",NaN,0
1,"[여름, 일렉트로니카, 매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일...","[여름, 일렉트로니카]","[매장음악, 취향저격, 댄스, 드라이브, 여행, 기분전환, 일렉, 신나는]",NaN,"[일렉트로닉, 일렉트로닉팝, ElectronicPop, electronica, tr...",NaN,0
2,"[잔잔한, 감성, 새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]","[잔잔한, 감성]","[새벽, 우울, 발라드, 알앤비, 기분전환, 사랑, 이별, 인디]",NaN,"[감성, 쓸쓸, 밤, 새벽감성, 새벽, 우울, 카페, 가을, 휴식, 혼자]",NaN,1
3,"[휴식, 새벽, 펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]","[휴식, 새벽]","[펍, 매장, 드라이브, 그루브, 라운지, 기분전환, 퇴근길, 세련된]",NaN,"[밤, 휴식, 잔잔한, 힐링, 가을, 비오는날, 산책, 사랑, 까페, 잠안올때]",NaN,0
4,"[잠, 매장, 매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]","[잠, 매장]","[매장음악, 카페, 잔잔한, 추억, 일상, 추위, 자장가, 새벽]",NaN,"[매장, 카페, 일상, 날씨, 잠안올때, 휴식, 생각, 잔잔한, 봄날, 편안한]",NaN,1


In [263]:
sum(svd_hit)/len(svd_hit)

0.5816278706452117

In [254]:
with open('./data/svd_knn_model.pickle', 'rb') as f:
    svd_p = pickle.load(f)

In [255]:
with open('./data/svd_tag_vectors.pickle', 'rb') as f:
    svd_vectors = pickle.load(f)

In [257]:
labels,distances = svd_p.knn_query(svd_vectors[3113], k = 11)

In [258]:
[id_to_tag[x] for x in labels[0][1:]]

['국내RnB', 'RnB소울', '가을노래', '겨울밤', '국내알앤비', 'R', '그냥', '폴킴', '겨울감성', '꿀성대']